In [ ]:
import altair as alt
import pandas as pd

# --- Cargar datos ---
horario = pd.read_csv("/content/Horario (1).csv")
horario.columns = horario.columns.str.strip().str.lower()

# --- Transformar formato ancho a largo ---
df_horario = horario.melt(id_vars='categoria', var_name='tipo_anio', value_name='cantidad')
df_horario['tipo'] = df_horario['tipo_anio'].str.extract(r'(^[a-z]+)')[0]
df_horario['anio'] = df_horario['tipo_anio'].str.extract(r'(\d{4})')[0].astype(int)

# --- Crear gráfico de líneas ---
chart_horario = alt.Chart(df_horario).mark_line(point=True).encode(
    x=alt.X('anio:O', title='Año'),
    y=alt.Y('cantidad:Q', title='Cantidad de accidentes'),
    color=alt.Color('categoria:N', title='Franja horaria', scale=alt.Scale(scheme='inferno')),
    strokeDash=alt.StrokeDash('tipo:N', title='Tipo de accidente'),
    tooltip=['categoria:N', 'anio:O', 'tipo:N', 'cantidad:Q']
).properties(
    title='Evolución de accidentes ferroviarios por horario y tipo (Chile, 2019–2024)',
    width=650,
    height=400
).configure_title(
    fontSize=18,
    anchor='start'
).configure_axis(
    labelFontSize=12,
    titleFontSize=13
)

# --- Exportar ---
chart_horario.save("horario.html")
chart_horario


alt.Chart(...)

In [ ]:
import altair as alt
import pandas as pd

# --- Cargar datos ---
dia = pd.read_csv("/content/Día.csv")
dia.columns = dia.columns.str.strip().str.lower()

# --- Transformar formato ancho a largo ---
df_dia = dia.melt(id_vars='categoria', var_name='tipo_anio', value_name='cantidad')
df_dia['tipo'] = df_dia['tipo_anio'].str.extract(r'(^[a-z]+)')[0]
df_dia['anio'] = df_dia['tipo_anio'].str.extract(r'(\d{4})')[0].astype(int)

# --- Agrupar por día y año ---
df_heat = df_dia.groupby(['categoria', 'anio'])['cantidad'].sum().reset_index()

# --- Crear mapa de calor ---
chart_dia = alt.Chart(df_heat).mark_rect().encode(
    x=alt.X('anio:O', title='Año'),
    y=alt.Y('categoria:N', title='Día de la semana'),
    color=alt.Color('cantidad:Q', title='Cantidad de accidentes', scale=alt.Scale(scheme='reds')),
    tooltip=['categoria:N', 'anio:O', 'cantidad:Q']
).properties(
    title='Concentración de accidentes ferroviarios por día y año (Chile, 2019–2024)',
    width=600,
    height=300
).configure_title(
    fontSize=18,
    anchor='start'
).configure_axis(
    labelFontSize=12,
    titleFontSize=13
)

# --- Exportar ---
chart_dia.save("dia.html")
chart_dia


alt.Chart(...)

In [52]:
import altair as alt
import pandas as pd

# --- Cargar datos ---
edad = pd.read_csv("/content/Edad (1).csv")
edad.columns = edad.columns.str.strip().str.lower()

# --- Transformar formato ancho a largo ---
df_long = edad.melt(id_vars='categoria', var_name='tipo_anio', value_name='cantidad')

# --- Extraer tipo de accidente y año ---
df_long['tipo'] = df_long['tipo_anio'].str.extract(r'(^[a-z]+)')[0]
df_long['anio'] = df_long['tipo_anio'].str.extract(r'(\d{4})')[0].astype(int)

# --- Completar valores faltantes con 0 ---
df_long['cantidad'] = df_long['cantidad'].fillna(0)

# --- Asegurar que existan todos los años entre 2019 y 2024 ---
categorias = df_long['categoria'].unique()
tipos = df_long['tipo'].unique()
años_completos = list(range(2019, 2025))

faltantes = []
for c in categorias:
    for t in tipos:
        for a in años_completos:
            if not ((df_long['categoria'] == c) & (df_long['tipo'] == t) & (df_long['anio'] == a)).any():
                faltantes.append({'categoria': c, 'tipo': t, 'anio': a, 'cantidad': 0})

# Agregar filas faltantes
df_long = pd.concat([df_long, pd.DataFrame(faltantes)], ignore_index=True)

# --- Etiqueta de estado ---
df_long['estado'] = df_long['cantidad'].apply(lambda x: 'Sin accidentes reportados' if x == 0 else 'Con registros')

# --- Crear gráfico ---
chart_edad = alt.Chart(df_long).mark_bar().encode(
    x=alt.X('anio:O', title='Año'),
    y=alt.Y('cantidad:Q', title='Cantidad de accidentes'),
    color=alt.condition(
        alt.datum.cantidad > 0,
        alt.Color('categoria:N', title='Rango de edad', scale=alt.Scale(scheme='viridis')),
        alt.value('#a6a6a6')  # gris para años sin registros
    ),
    column=alt.Column('tipo:N', title='Tipo de accidente',
                      header=alt.Header(titleOrient="bottom", labelOrient="bottom")),
    tooltip=[
        alt.Tooltip('categoria:N', title='Rango de edad'),
        alt.Tooltip('anio:O', title='Año'),
        alt.Tooltip('tipo:N', title='Tipo de accidente'),
        alt.Tooltip('cantidad:Q', title='Cantidad'),
        alt.Tooltip('estado:N', title='Estado del registro')
    ]
).properties(
    title='Evolución de accidentes ferroviarios por edad y tipo (Chile, 2019–2024)\n*Si no hay color, indica años sin registros oficiales.*',
    width=150,
    height=300
).configure_title(
    fontSize=18,
    anchor='start'
).configure_axis(
    labelFontSize=12,
    titleFontSize=13
)

# --- Exportar ---
chart_edad.save("edad.html")
chart_edad


alt.Chart(...)